# Testing Model (C)

In [1]:
import os
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
import numpy as np
from torchvision import transforms, models

# Check if CUDA is available and set the device accordingly
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define Precision and Recall functions
def calculate_precision(pred, target):
    true_positive = torch.sum(pred * target)
    predicted_positive = torch.sum(pred)
    return (true_positive + 1e-6) / (predicted_positive + 1e-6)

def calculate_recall(pred, target):
    true_positive = torch.sum(pred * target)
    actual_positive = torch.sum(target)
    return (true_positive + 1e-6) / (actual_positive + 1e-6)

# IoU and Dice coefficient functions
def calculate_iou(pred, target):
    intersection = torch.sum(pred * target)
    union = torch.sum(pred) + torch.sum(target) - intersection
    return (intersection + 1e-6) / (union + 1e-6)

def calculate_dice(pred, target):
    intersection = torch.sum(pred * target)
    return (2 * intersection + 1e-6) / (torch.sum(pred) + torch.sum(target) + 1e-6)


# Function to evaluate the model
def evaluate_model(model, test_img_path, test_mask_path):
    model.eval()

    # Transform for test images
    transform = transforms.Compose([
        transforms.Resize((640, 640)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])

    image_names = sorted(os.listdir(test_img_path))
    mask_names = sorted(os.listdir(test_mask_path))

    precision_list, recall_list, iou_list, dice_list = [], [], [], []

    with torch.no_grad():
        for img_name, mask_name in zip(image_names, mask_names):
            # Load image and mask
            img_path = os.path.join(test_img_path, img_name)
            mask_path = os.path.join(test_mask_path, mask_name)

            image = Image.open(img_path).convert('RGB')
            mask = Image.open(mask_path).convert('L')

            image = transform(image).unsqueeze(0).to(device)
            mask = transforms.ToTensor()(mask).to(device)
            mask = torch.where(mask > 0, torch.tensor(1.0).to(device), torch.tensor(0.0).to(device))

            # Get model prediction
            output = model(image)
            pred = torch.where(output > 0.5, torch.tensor(1.0).to(device), torch.tensor(0.0).to(device))

            # Calculate metrics
            precision = calculate_precision(pred, mask).item()
            recall = calculate_recall(pred, mask).item()
            iou = calculate_iou(pred, mask).item()
            dice = calculate_dice(pred, mask).item()

            precision_list.append(precision)
            recall_list.append(recall)
            iou_list.append(iou)
            dice_list.append(dice)

    # Calculate mean metrics
    mean_precision = np.mean(precision_list)
    mean_recall = np.mean(recall_list)
    mean_iou = np.mean(iou_list)
    mean_dice = np.mean(dice_list)

    print(f"Mean Precision: {mean_precision:.4f}")
    print(f"Mean Recall: {mean_recall:.4f}")
    print(f"Mean IoU: {mean_iou:.4f}")
    print(f"Mean Dice: {mean_dice:.4f}")

    return mean_precision, mean_recall, mean_iou, mean_dice


# Define U-Net Model with Pretrained ResNet Encoder
class UNetWithResNetEncoder(nn.Module):
    def __init__(self, out_channels=1):
        super(UNetWithResNetEncoder, self).__init__()

        # Pretrained ResNet backbone
        resnet = models.resnet34(pretrained=True)

        # Encoder layers from ResNet
        self.enc1 = nn.Sequential(resnet.conv1, resnet.bn1, resnet.relu)  # First conv block
        self.enc2 = nn.Sequential(resnet.layer1)  # ResNet layer 1
        self.enc3 = nn.Sequential(resnet.layer2)  # ResNet layer 2
        self.enc4 = nn.Sequential(resnet.layer3)  # ResNet layer 3
        self.enc5 = nn.Sequential(resnet.layer4)  # ResNet layer 4

        # Decoder layers
        def up_conv(in_channels, out_channels):
            return nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2)

        def conv_block(in_channels, out_channels):
            block = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
                nn.ReLU(inplace=True),
                nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
                nn.ReLU(inplace=True)
            )
            return block

        self.upconv4 = up_conv(512, 256)
        self.dec4 = conv_block(512, 256)

        self.upconv3 = up_conv(256, 128)
        self.dec3 = conv_block(256, 128)

        self.upconv2 = up_conv(128, 64)
        self.dec2 = conv_block(128, 64)

        self.upconv1 = up_conv(64, 64)
        self.dec1 = conv_block(64 + 64, 64)

        # Output layer
        self.conv_last = nn.Conv2d(64, out_channels, kernel_size=1)

    def forward(self, x):
        # Encoder
        enc1 = self.enc1(x)
        enc2 = self.enc2(enc1)
        enc3 = self.enc3(enc2)
        enc4 = self.enc4(enc3)
        enc5 = self.enc5(enc4)

        # Decoder
        dec4 = self.upconv4(enc5)
        dec4 = torch.cat((dec4, self._align_tensor(enc4, dec4)), dim=1)
        dec4 = self.dec4(dec4)

        dec3 = self.upconv3(dec4)
        dec3 = torch.cat((dec3, self._align_tensor(enc3, dec3)), dim=1)
        dec3 = self.dec3(dec3)

        dec2 = self.upconv2(dec3)
        dec2 = torch.cat((dec2, self._align_tensor(enc2, dec2)), dim=1)
        dec2 = self.dec2(dec2)

        dec1 = self.upconv1(dec2)
        dec1 = torch.cat((dec1, self._align_tensor(enc1, dec1)), dim=1)
        dec1 = self.dec1(dec1)

        return torch.sigmoid(self.conv_last(dec1))

    def _align_tensor(self, enc, dec):
        """Align encoder tensor to match the size of the decoder tensor."""
        enc_h, enc_w = enc.size(2), enc.size(3)
        dec_h, dec_w = dec.size(2), dec.size(3)

        if enc_h != dec_h or enc_w != dec_w:
            enc = nn.functional.interpolate(enc, size=(dec_h, dec_w), mode='bilinear', align_corners=False)

        return enc


# Paths to test images and masks
test_img_path = '/home/tim/Documents/06_Projekt_RainyNuScenes/rainynuscenes/data/rainyNuScenes/dataset_split/train/images'
test_mask_path = '/home/tim/Documents/06_Projekt_RainyNuScenes/rainynuscenes/data/rainyNuScenes/dataset_split/train/labels'
save_path = '/home/tim/Documents/06_Projekt_RainyNuScenes/rainynuscenes/detection_application/results/rainy_nuscenes_model.pth'


# Load the trained model
model = UNetWithResNetEncoder(out_channels=1).to(device)
model.load_state_dict(torch.load(save_path, map_location=device))

# Evaluate the model
print("RainyNuScenes")
mean_precision, mean_recall, mean_iou, mean_dice = evaluate_model(model, test_img_path, test_mask_path)




/home/tim/Documents/06_Projekt_RainyNuScenes/rainynuscenes/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/tim/Documents/06_Projekt_RainyNuScenes/rainynuscenes/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/tmp/ipykernel_181219/3804334894.py:181: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unp

RainyNuScenes
Mean Precision: 0.7755
Mean Recall: 0.6762
Mean IoU: 0.5311
Mean Dice: 0.6697


In [2]:
# Paths to test images and masks
test_img_path = '/home/tim/Documents/06_Projekt_RainyNuScenes/rainynuscenes/data/soiling_woodscape_data/test/droplet_rgb'
test_mask_path = '/home/tim/Documents/06_Projekt_RainyNuScenes/rainynuscenes/data/soiling_woodscape_data/test/droplet_masks'


# Load the trained model
model = UNetWithResNetEncoder(out_channels=1).to(device)
model.load_state_dict(torch.load(save_path, map_location=device))

# Evaluate the model
print("WoodScape")
mean_precision, mean_recall, mean_iou, mean_dice = evaluate_model(model, test_img_path, test_mask_path)


/tmp/ipykernel_181219/2581559796.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(save_path, map_location=device))


WoodScape
Mean Precision: 0.8501
Mean Recall: 0.3876
Mean IoU: 0.3566
Mean Dice: 0.4970
